In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
df = pd.read_csv('sample_data.csv')
df.head(2)

,Unnamed: 0,user_id,restaurant_id,Rating_x,Unnamed: 0_y,resId,title,No_of_Reviews,Tags,Details,Range_n_Cuisine
0,0,0,363,0.0,363,363,Chartreuse Moose Cappuccino Bar & Bistro,170.0,"wraps , breakfast wrap , beans , great coffee...","{'CUISINES': 'Cafe, Canadian', 'Special Diets'...","['$$ - $$$', 'Quick Bites', 'Cafe', 'Canadian']"
1,1,0,363,0.0,363,363,Chartreuse Moose Cappuccino Bar & Bistro,170.0,"wraps , breakfast wrap , beans , great coffee...","{'CUISINES': 'Cafe, Canadian', 'Special Diets'...","['$$ - $$$', 'Quick Bites', 'Cafe', 'Canadian']"


In [4]:
df1 = df.copy()
df1 = df1.drop(['Unnamed: 0','Unnamed: 0_y','resId'], axis = 1)
df1.head(1)

,user_id,restaurant_id,Rating_x,title,No_of_Reviews,Tags,Details,Range_n_Cuisine
0,0,363,0.0,Chartreuse Moose Cappuccino Bar & Bistro,170.0,"wraps , breakfast wrap , beans , great coffee...","{'CUISINES': 'Cafe, Canadian', 'Special Diets'...","['$$ - $$$', 'Quick Bites', 'Cafe', 'Canadian']"


In [5]:
df1['num_of_reviews_by_user'] = df1.groupby(['restaurant_id'])['user_id'].transform('count')
df1['num_of_reviews_for_restaurant'] = df1.groupby(['user_id'])['restaurant_id'].transform('count')
print("Maximum number of reviews by user : " + str(df1['num_of_reviews_by_user'].max()))
print("Maximum number of reviews per restaurant : " + str(df1['num_of_reviews_for_restaurant'].max()))

Maximum number of reviews by user : 1176
Maximum number of reviews per restaurant : 362


In [6]:
df1.restaurant_id.value_counts()

10     1176
217     638
41      510
225     266
204      64
363      54
367      18
368      14
370      10
376       7
379       6
373       6
380       6
369       5
364       4
Name: restaurant_id, dtype: int64

In [7]:
df1.Tags.value_counts().sum()

214

In [8]:
df1.Range_n_Cuisine.value_counts().sum()

2784

In [9]:
df1.Details.value_counts().sum()

2784

In [10]:
df2 = df1.copy()
df2 = df2[['restaurant_id', 'Tags']]
df2.shape

(2784, 2)

In [11]:
# dummify categorical features
restr_metadata = pd.get_dummies(df2, columns = ['Tags'])

restr_metadata = restr_metadata.sort_values('restaurant_id').reset_index().drop('index', axis=1)
restr_metadata.shape

(2784, 16)

In [12]:
from scipy.sparse import csr_matrix
# convert to csr matrix
restr_metadata_csr = csr_matrix(restr_metadata.drop('restaurant_id', axis=1).values)
restr_metadata_csr

<2784x15 sparse matrix of type '<class 'numpy.uint8'>'
	with 214 stored elements in Compressed Sparse Row format>

In [13]:
user_restr_interaction = pd.pivot_table(df1, index='user_id', columns='restaurant_id', values='Rating_x')

# fill missing values with 0
user_restr_interaction = user_restr_interaction.fillna(0)

user_restr_interaction.head()

restaurant_id,10,41,204,217,225,363,364,367,368,369,370,373,376,379,380
user_id,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,1.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0
1,0.0,0.0,2.5,0.00,0.0,0.0,0.0,0.0,3.5,0.0,1.5,0.0,4.5,0.0,0.0
2,2.0,0.0,0.0,0.00,0.0,1.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,4.0,0.0,0.00,4.5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.5,0.0,0.0,2.75,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0


In [14]:
user_restr_interaction.shape

(20, 15)

In [15]:
user_id = list(user_restr_interaction.index)
user_dict = {}
counter = 0 
for i in user_id:
    user_dict[i] = counter
    counter += 1

In [16]:
item_dict ={}
df = df1[['restaurant_id', 'title']].sort_values('restaurant_id').reset_index()

for i in range(df.shape[0]):
    item_dict[(df.loc[i,'restaurant_id'])] = df.loc[i,'title']

In [17]:
# convert to csr matrix
from scipy.sparse import csr_matrix
user_restr_interaction_csr = csr_matrix(user_restr_interaction.values)
user_restr_interaction_csr

<20x15 sparse matrix of type '<class 'numpy.float64'>'
	with 87 stored elements in Compressed Sparse Row format>

In [18]:
from lightfm import LightFM
model = LightFM(loss='warp',
                random_state=2016,
                learning_rate=0.90,
                no_components=150,
                user_alpha=0.000005)

model = model.fit(user_restr_interaction_csr,
                  epochs=100,
                  num_threads=16, verbose=False)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [19]:
def sample_recommendation_user(model, interactions, user_id, user_dict, 
                               item_dict,threshold = 0,nrec_items = 5, show = True):
    
    n_users, n_items = interactions.shape
    user_x = user_dict[user_id]
    scores = pd.Series(model.predict(user_x,np.arange(n_items), item_features=restr_metadata_csr))
    scores.index = interactions.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    
    known_items = list(pd.Series(interactions.loc[user_id,:] \
                                 [interactions.loc[user_id,:] > threshold].index).sort_values(ascending=False))
    
    scores = [x for x in scores if x not in known_items]
    return_score_list = scores[0:nrec_items]
    known_items = list(pd.Series(known_items).apply(lambda x: item_dict[x]))
    scores = list(pd.Series(return_score_list).apply(lambda x: item_dict[x]))
    if show == True:
        print ("User: " + str(user_id))
        print("Known Likes:")
        counter = 1
        for i in known_items:
            print(str(counter) + '- ' + i)
            counter+=1

        print("\n Recommended Items:")
        counter = 1
        for i in scores:
            print(str(counter) + '- ' + i)
            counter+=1

In [20]:

sample_recommendation_user(model, user_restr_interaction, 7, user_dict, item_dict)

User: 7
Known Likes:
1- The Great Wok
2- Jake's
3- Chartreuse Moose Cappuccino Bar & Bistro
4- A&W Restaurant
5- Canadian 2 For 1 Pizza

 Recommended Items:
1- Smitty's 100 Mile House
2- El Caballo Restaurant
3- P Bass Fish and Chips
4- Blue Sky Chinese Restaurant
5- Farrier pub
